# Find weather stations that set flags
From the AWS data catalog of station data, find the subset of weather stations that set flags 
<br>Output a DataFrame

In [ ]:
import xarray as xr
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

## 1. Define some helper functions

In [ ]:
def find_flags(station_ds):
    """Find unique flags in a Dataset.
    Filters through flag variables; assumes flag variables contain the substring '_eraqc'

    Parameters
    ----------
    station_ds: xr.Dataset
        Dataset containing station data, with each variable as a unique data variable

    Returns
    -------
    unique_flags: list or None
        List of unique flag values found in station_ds
        Returns None if no flags found for any variable

    """

    # Get the string names of the eraqc flag variables
    era_flag_var_names = [var for var in station_ds.data_vars if "_eraqc" in var]

    # Subset Dataset to just contain flag variables
    station_flags_ds = station_ds[era_flag_var_names]

    # Check for nulls in the flag variables
    # If all are null, that means the station set no flags for this event!
    all_null = station_flags_ds.to_array().isnull().all().item()
    unique_flags = None

    # If there are some flags set, find out what they are
    if not all_null:
        # Stack all the variables, since we don't care which variables the flags belong to
        # Makes it easier to search the array (fewer lines of code :)
        # Works like np.flatten, but on an xarray object
        stacked = station_flags_ds.to_array().stack(
            {"everything": ["variable", "time"]}
        )

        # Drop all non-null values
        # i.e. [nan, nan, 23, nan, 17] --> [23, 17]
        all_flags = stacked.where(~stacked.isnull(), drop=True)

        # Get unique flag values as integers
        # nan is treated as a float so the .isnull() step converted floats to ints (I think)
        # i.e. [23.0, 23.0, 23.0, 17.0, 23.0, 19.0, 19.0] --> [23, 17, 19]
        unique_flags = list(np.unique(all_flags.values).astype(int))

    return unique_flags

## 2. Read in county shapefile and station metadata csv
County shapefile is used for getting event geometry <br>Station metadata is used to get the station ids to read in station data from the S3 bucket

In [ ]:
# Read in a list of weather stations
# We just use the names of the stations to filter
# Ideally I'd like to see this moved to the AWS bucket
train_stns = pd.read_csv("../data/qaqc_training_station_list.csv")

# Create new geometry column from lat and lon columns
# This will enable us to filter through the GeoDataFrame using our county geometry
train_stns = gpd.GeoDataFrame(
    train_stns,
    geometry=gpd.points_from_xy(
        train_stns.longitude, train_stns.latitude, crs="EPSG:4326"
    ),
)

In [ ]:
# Read in county shapefiles from S3
ca_counties = gpd.read_file(
    "s3://wecc-historical-wx/0_maps/ca_counties/CA_Counties.shp"
)
ca_counties = ca_counties.to_crs(train_stns.crs)  # Convert to station CRS
# ca_counties.head()

In [ ]:
# Plot all the stations and all the counties
# This is a check to ensure both have the same projection
fig, ax = plt.subplots()
ca_counties.boundary.plot(color="black", ax=ax, linewidth=0.75, zorder=10)
train_stns.geometry.plot(markersize=0.9, ax=ax, zorder=15);

## 3. Set your event date and (optional) event geometry
For example, lets use the Montecito mudslide from January 5-9, 2018

In [ ]:
event_start_date = "2018-01-01"  # Start a few days before the mudslide
event_end_date = "2018-01-09"

event_geom = ca_counties[ca_counties["NAME"] == "Santa Barbara"]  # Get event geometry

## 4. Get the stations that intersect the event geometry

In [ ]:
train_stns["intersects"] = train_stns.intersects(
    event_geom.unary_union
)  # See which stations intersect with the event polygon
event_stns = train_stns[train_stns["intersects"] == True].reset_index(
    drop=True
)  # Get just those stations, drop the others
event_stns

In [ ]:
# Make a plot to visualize the stations and event poly
fig, ax = plt.subplots()
event_geom.boundary.plot(color="black", ax=ax, zorder=10)
event_stns.geometry.plot(markersize=15, ax=ax, zorder=15);

## 5. Filter through stations to see if flags were set

In [ ]:
# Flags, network, and station ID will be stored here
# Dictionary can be easily convered to a DataFrame and output as a csv
stations_with_flags = {"network": [], "era-id": [], "flags": []}

# Loop through each station to look for flags
for i in tqdm(range(len(event_stns))):
    # Get info for one station
    network_name, station_id = event_stns.iloc[i][["network", "era-id"]]

    station_ds = xr.open_zarr(f"s3://wecc-historical-wx/3_qaqc_wx/{network_name}/{station_id}.zarr").compute()

    # Reduce dimension of object
    # "station" is a singleton dimension
    station_ds = station_ds.squeeze()

    # Subset Dataset to event time period
    station_ds = station_ds.sel(time=slice(event_start_date, event_end_date))

    # Look for flags
    # If no flags found, find_flags returns None
    unique_flags = find_flags(station_ds)

    # If flags are found, save them to the dictionary
    if unique_flags is not None:
        stations_with_flags["network"].append(network_name)
        stations_with_flags["era-id"].append(station_id)
        stations_with_flags["flags"].append(unique_flags)

In [ ]:
# Output a DataFrame
# Note: No record which variables these flags belong to
stations_with_flags_df = pd.DataFrame(stations_with_flags)
stations_with_flags_df